In [2]:
!pip install -qU "autogen-agentchat" "autogen-ext[openai,azure]"

In [8]:
!pip install yfinance

In [5]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [14]:
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

In [20]:
# Define a tool
async def get_stock_data(symbol1: str, symbol2: str) -> dict:
  """Use this function to get the stock data for the past month."""
  end_date = datetime.now()
  start_date = end_date - timedelta(days=30)

  # Fetch historical data for the past month
  stock1 = yf.Ticker(symbol1)
  stock2 = yf.Ticker(symbol2)

  hist_stock1 = stock1.history(start=start_date, end=end_date)
  hist_stock2 = stock2.history(start=start_date, end=end_date)

  # Fetch live price (the last available closing price)
  live_price_stock1 = hist_stock1['Close'][-1]
  live_price_stock2 = hist_stock2['Close'][-1]

  return {
      symbol1: {
          'live_price': live_price_stock1,
          'historical_data': hist_stock1
      },
      symbol2: {
          'live_price': live_price_stock2,
          'historical_data': hist_stock2
      }
  }



async def main() -> None:

    user_proxy = AssistantAgent(
        name="user_proxy",
        system_message="""You are a human admin tasked with the following:

        1. Fetch Real-time Stock Data using stock_price_agent
        2. Generate Investment Comparison using investment_comparison_agent
        3. Risk Assessment using risk_assessment
        4. Generate Investment Advice using investment_advice_agent
        5. Display Results: Show the final generated report and the investment advice.
        """, #Show the final generated report on a user-friendly interface with data visualizations (graphs, tables) and the investment advice. use plot_agent for creating any charts/plots on the data.
        model_client=OpenAIChatCompletionClient(
            model="gpt-4o",
            api_key="OPEN_AI_API_KEY",
        )
    )

    # Define an agent
    stock_price_agent = AssistantAgent(
        name="stock_price_agent",
        model_client=OpenAIChatCompletionClient(
            model="gpt-4o",
            api_key="OPEN_AI_API_KEY",
        ),
        tools=[get_stock_data],
    )

    investment_comparison_agent = AssistantAgent(
        name="investment_comparison_agent",
        system_message="""Your task is to generate an **investment comparison** between two stock price data based on key financial metrics.

        Requirements:
        1. **Current Stock Price** – Retrieve and display the latest market price for both stocks.
        2. **Historical Performance** – Compare stock price trends over a specified period (e.g., 1 months,).
        3. **Market Trends & Volatility** – Analyze recent market movements and stock volatility.
        4. **Analyst Ratings & Sentiment** – Summarize expert opinions, buy/sell recommendations, and overall market sentiment.
        5. **Key Financial Indicators** – Evaluate fundamental metrics such as P/E ratio, dividend yield, and revenue growth.

        Output: Provide a structured **side-by-side comparison** of both stocks.\n""- Use **clear financial insights** to highlight potential investment opportunities or risks.

        """,
        model_client=OpenAIChatCompletionClient(
            model="gpt-4o",
            api_key="OPEN_AI_API_KEY",
        ))

    risk_assessment = AssistantAgent(
        name="risk_assessment_agent",
        system_message="""You are a financial risk analysis expert. Your task is to analyze the risk of a given stock based on its historical price data and volatility.

You will receive stock price data, including historical closing prices for the past month and the latest live price. Based on this data, assess the stock's volatility, calculate its standard deviation, and determine its risk level (low, medium, or high).

Consider the following factors in your analysis:

Volatility: Calculate the standard deviation of daily returns.
Trend Analysis: Identify whether the stock is trending upwards, downwards, or sideways.
Recent Performance: Compare the latest price to the average price over the past month.
Risk Categorization: Define risk levels as follows:
Low risk: Standard deviation < 1%
Medium risk: Standard deviation between 1% and 3%
High risk: Standard deviation > 3%
Provide a clear and concise risk assessment along with a summary of your findings.""",
        model_client=OpenAIChatCompletionClient(
            model="gpt-4o",
            api_key="OPEN_AI_API_KEY",
        ))

    investment_advice = AssistantAgent(
        name="investment_advice_agent",
        system_message="create a detailed report comparing the stocks data, including risk analysis and a recommendation on which stock is a better investment based on recent data.",
        model_client=OpenAIChatCompletionClient(
            model="gpt-4o",
            api_key="OPEN_AI_API_KEY",
        ))

    plot_agent = AssistantAgent(
        name="plot_agent",
        system_message="Return a plot based on the stock price data provided to you.",
        model_client=OpenAIChatCompletionClient(
            model="gpt-4o",
            api_key="OPEN_AI_API_KEY",
        ))

    # Define a team with a single agent and maximum auto-gen turns of 1.
    agent_team = RoundRobinGroupChat([user_proxy, stock_price_agent, investment_comparison_agent, risk_assessment, investment_advice], max_turns=5)

    while True:
        # Get user input from the console.
        user_input = input("Enter a message (type 'exit' to leave): ")
        if user_input.strip().lower() == "exit":
            break
        # Run the team and stream messages to the console.
        stream = agent_team.run_stream(task=user_input)
        await Console(stream)


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

Enter a message (type 'exit' to leave): META, TSLA
---------- user ----------
META, TSLA
---------- user_proxy ----------
To provide you with accurate investment information and advice regarding META (Meta Platforms, Inc.) and TSLA (Tesla, Inc.), I'll follow the steps outlined:

### Step 1: Fetch Real-time Stock Data

Let's start by fetching the current stock prices for both META and TSLA using the stock_price_agent.

### Step 2: Generate Investment Comparison

Next, we'll compare the financial metrics and performance over time for these two companies to provide a comprehensive comparison.

### Step 3: Risk Assessment

We'll then perform a risk assessment to understand the associated risks involved in investing in META and TSLA.

### Step 4: Generate Investment Advice

Based on the performance data and risk assessment, we'll generate personalized investment advice.

### Step 5: Display Results

Finally, I'll display the consolidated results, including the final report and investment ad

<ipython-input-20-8d591b4a958f>:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  live_price_stock1 = hist_stock1['Close'][-1]
<ipython-input-20-8d591b4a958f>:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  live_price_stock2 = hist_stock2['Close'][-1]


---------- stock_price_agent ----------
[FunctionCall(id='call_TxLxIuMpOAKs7vT9fVmjbJFQ', arguments='{"symbol1": "META", "symbol2": "TSLA"}', name='get_stock_data')]
---------- stock_price_agent ----------
[FunctionExecutionResult(content="{'META': {'live_price': 674.3300170898438, 'historical_data':                                  Open        High         Low       Close  \\\nDate                                                                        \n2024-12-30 00:00:00-05:00  588.750000  596.940002  585.580017  591.239990   \n2024-12-31 00:00:00-05:00  592.270020  593.969971  583.849976  585.510010   \n2025-01-02 00:00:00-05:00  589.719971  604.909973  587.820007  599.239990   \n2025-01-03 00:00:00-05:00  604.760010  609.500000  596.409973  604.630005   \n2025-01-06 00:00:00-05:00  611.830017  630.989990  605.619995  630.200012   \n2025-01-07 00:00:00-05:00  631.700012  632.099976  608.229980  617.890015   \n2025-01-08 00:00:00-05:00  613.400024  616.440002  602.789978  610.719971